# Explore Data

In [1]:
import pandas as pd
import numpy as np
#%pip install networkx
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [9]:
pd.set_option('display.max_columns', None)

business problem:

* to find some influential players, and it also means they could influence some decision in league. Furthermore, some advertisement cooperation about sports could begin with them.

* time series analysis about the statistics of player, like shoot, and use it as prediction of future trend of game or just for a rookie to help him accommodate NBA. More deeply, it's expected that mid range shoot is disappearing, we could apply data to find the reason, like lower reward or lower percentage. For more we could use cluster method to divide all players into serval groups, the investigate the difference performance in mid-range to show that if that really disappear in star player or low-level player.

Possible work:

* simple EDA(or not, actually we don't need it), data integration

* network building, visualization(this part need tha graph simple, instead of full of edges) and network analysis based on those attributes.

* nlp extraction about the shooting information, apply regex.

* time-series to find some trend of pattern

* clustering and investigate the difference among groups in some fields, shooting, steal, block. clustering attribute could be average points, avg pct

* for time-series part, we could select some players to compare them with their model player in draft

* select some star players to build time-series data, and look if there are some strange performance season.

possible work division:

data integration & network, this job could be done by one alone

nlp extraction & clustering, clustering should be done after time-series

time-series & particular player time series analysis, all these part should be after nlp extraction the feature of player action



In [10]:
# data from https://sports-statistics.com/sports-data/nba-basketball-datasets-csv-files/
pbp = pd.read_csv("./2000-01_pbp.csv")
pbp.drop(pbp.columns[0],axis=1,inplace=True)
pbp.head(100)
# for description column, based on nlp, extract the data of miss/make, type of score, distance for shooting time series

,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER1_TEAM_CITY,PLAYER1_TEAM_ID,PLAYER1_TEAM_NICKNAME,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER2_TEAM_CITY,PLAYER2_TEAM_ID,PLAYER2_TEAM_NICKNAME,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING
0,0,12,0,20001116,NaN,NaN,12:00,1,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:11 PM
1,0,10,1,20001116,Jump Ball Grant vs. Mutombo: Tip to Hill,NaN,12:00,1,4.0,5,5,258,Brian Grant,MIA,Miami,1.610613e+09,Heat,87,Dikembe Mutombo,PHI,Philadelphia,1.610613e+09,76ers,238,Tyrone Hill,PHI,Philadelphia,1.610613e+09,76ers,NaN,NaN,NaN,12:12 PM
2,1,1,2,20001116,NaN,NaN,11:45,1,5.0,0,0,947,Allen Iverson,PHI,Philadelphia,1.610613e+09,76ers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2 - 0,-2,Iverson 21' Jump Shot (2 PTS),12:13 PM
3,1,6,3,20001116,NaN,NaN,11:28,1,5.0,0,1,238,Tyrone Hill,PHI,Philadelphia,1.610613e+09,76ers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hill P.FOUL (P1.T1),12:13 PM
4,1,1,4,20001116,Mason 17' Jump Shot (2 PTS),NaN,11:19,1,4.0,0,0,193,Anthony Mason,MIA,Miami,1.610613e+09,Heat,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2 - 2,TIE,NaN,12:13 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,1,99,20001116,Bowen 24' 3PT Jump Shot (3 PTS) (Mourning 1 AST),NaN,10:46,2,4.0,4,0,1477,Bruce Bowen,MIA,Miami,1.610613e+09,Heat,297,Alonzo Mourning,MIA,Miami,1.610613e+09,Heat,0,NaN,NaN,NaN,NaN,NaN,19 - 26,7,NaN,12:40 PM
96,42,2,100,20001116,Mourning BLOCK (1 BLK),NaN,10:18,2,5.0,0,4,243,Aaron McKie,PHI,Philadelphia,1.610613e+09,76ers,0,NaN,NaN,NaN,NaN,NaN,297,Alonzo Mourning,MIA,Miami,1.610613e+09,Heat,NaN,NaN,MISS McKie Driving Layup,12:40 PM
97,0,4,101,20001116,Mourning REBOUND (Off:0 Def:2),NaN,10:17,2,4.0,0,0,297,Alonzo Mourning,MIA,Miami,1.610613e+09,Heat,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:40 PM
98,1,2,102,20001116,MISS Mason 12' Jump Shot,NaN,10:01,2,4.0,0,0,193,Anthony Mason,MIA,Miami,1.610613e+09,Heat,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:41 PM


In [24]:
pbp["GAME_ID"].unique()

array([20001116, 20000520, 20000629, ..., 20001060, 20000516, 20000744])

In [8]:
pbp.columns

Index(['EVENTMSGACTIONTYPE', 'EVENTMSGTYPE', 'EVENTNUM', 'GAME_ID',
       'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION', 'PCTIMESTRING', 'PERIOD',
       'PERSON1TYPE', 'PERSON2TYPE', 'PERSON3TYPE', 'PLAYER1_ID',
       'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION', 'PLAYER1_TEAM_CITY',
       'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_NICKNAME', 'PLAYER2_ID',
       'PLAYER2_NAME', 'PLAYER2_TEAM_ABBREVIATION', 'PLAYER2_TEAM_CITY',
       'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_NICKNAME', 'PLAYER3_ID',
       'PLAYER3_NAME', 'PLAYER3_TEAM_ABBREVIATION', 'PLAYER3_TEAM_CITY',
       'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_NICKNAME', 'SCORE', 'SCOREMARGIN',
       'VISITORDESCRIPTION', 'WCTIMESTRING'],
      dtype='object')

In [11]:
# data from https://www.kaggle.com/datasets/nathanlauga/nba-games/
games = pd.read_csv("./archive/games.csv")
detail = pd.read_csv("./archive/games_details.csv")
players = pd.read_csv("./archive/players.csv")
ranking = pd.read_csv("./archive/ranking.csv")

/var/folders/yc/2xfv1_bx5p1bsggrm1jp8ljc0000gn/T/ipykernel_33780/3474592753.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  detail = pd.read_csv("./archive/games_details.csv")


In [12]:
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,0.457,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,0.313,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,0.297,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,0.378,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0


In [13]:
detail.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,18:06,1.0,1.0,1.000,0.0,0.0,0.000,0.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,31:01,7.0,14.0,0.500,2.0,4.0,0.500,7.0,10.0,0.7,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,21:42,6.0,9.0,0.667,0.0,0.0,0.000,1.0,1.0,1.0,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,30:20,4.0,13.0,0.308,1.0,6.0,0.167,1.0,1.0,1.0,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,27:44,7.0,12.0,0.583,1.0,3.0,0.333,4.0,4.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0


In [21]:
detail.sort_values(by="GAME_ID", ascending=True)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
306920,10300001,1610612742,DAL,Dallas,696,Travis Best,NaN,NaN,NaN,5,4.0,7.0,0.571,0.0,0.0,0.0,2.0,4.0,0.5,0.0,1.0,1.0,5.0,0.0,0.0,3.0,6.0,10.0,NaN
306918,10300001,1610612742,DAL,Dallas,959,Steve Nash,NaN,NaN,NaN,5,0.0,2.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,5.0,1.0,0.0,2.0,3.0,0.0,NaN
306891,10300001,1610612762,UTA,Utah,2144,Ademola Okulaja,NaN,NaN,NaN,5,1.0,1.0,1.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,2.0,NaN
306892,10300001,1610612762,UTA,Utah,1905,Andrei Kirilenko,NaN,NaN,NaN,5,0.0,5.0,0.000,0.0,3.0,0.0,1.0,2.0,0.5,0.0,5.0,5.0,2.0,2.0,1.0,4.0,1.0,1.0,NaN
306893,10300001,1610612762,UTA,Utah,1052,Ben Handlogten,NaN,NaN,NaN,5,2.0,4.0,0.500,0.0,0.0,0.0,0.0,2.0,0.0,2.0,4.0,6.0,0.0,0.0,0.0,2.0,4.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17064,52100211,1610612740,NOP,New Orleans,1626195,Willy Hernangomez,Willy,NaN,DNP - Coach's Decision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17063,52100211,1610612740,NOP,New Orleans,203503,Tony Snell,Tony,NaN,NaN,3:48,0.0,1.0,0.000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,-9.0
17075,52100211,1610612746,LAC,LA,1630527,Brandon Boston Jr.,Brandon,NaN,DNP - Coach's Decision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17073,52100211,1610612746,LAC,LA,203496,Robert Covington,Robert,NaN,NaN,34:44,4.0,7.0,0.571,2.0,4.0,0.5,4.0,5.0,0.8,0.0,3.0,3.0,1.0,1.0,0.0,0.0,5.0,14.0,0.0


In [22]:
games[games["GAME_ID"]==10300001]

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
19288,2003-10-05,10300001,Final,1610612762,1610612742,2003,1610612762,90.0,0.457,0.735,0.143,23.0,41.0,1610612742,85.0,0.447,0.5,0.25,20.0,38.0,1


In [14]:
players.head()

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,Royce O'Neale,1610612762,1626220,2019
1,Bojan Bogdanovic,1610612762,202711,2019
2,Rudy Gobert,1610612762,203497,2019
3,Donovan Mitchell,1610612762,1628378,2019
4,Mike Conley,1610612762,201144,2019


In [23]:
ranking.head(100)

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,NaN
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,NaN
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9,NaN
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1610612757,0,22022,2022-12-16,West,Portland,29,16,13,0.552,7-6,9-7,NaN
96,1610612746,0,22022,2022-12-16,West,LA Clippers,31,17,14,0.548,9-7,8-7,NaN
97,1610612762,0,22022,2022-12-16,West,Utah,31,17,14,0.548,11-5,6-9,NaN
98,1610612742,0,22022,2022-12-16,West,Dallas,29,15,14,0.517,12-5,3-9,NaN
